# csv 데이터 불러오기

In [1]:
import pandas as pd

input_file = pd.read_csv('train_data0307.csv')
target_file = pd.read_csv('target_data0307.csv')

# dynamic 추출 함수

In [2]:
def distribute_dynamic(mean_velocity):
    """
    velocity에 따른 dynamic 기호 추출
    """
    dynamic = ''
    if 0 <= mean_velocity < 36:
        dynamic = 'ppp'
    elif 36 <= mean_velocity < 49:
        dynamic = 'pp'
    elif 49 <= mean_velocity < 62:
        dynamic = 'p'
    elif 62 <= mean_velocity < 75:
        dynamic = 'mp'
    elif 75 <= mean_velocity < 88:
        dynamic = 'mf'
    elif 88 <= mean_velocity < 101:
        dynamic = 'f'
    elif 101 <= mean_velocity < 114:
        dynamic = 'ff'
    elif 114 <= mean_velocity <= 127:
        dynamic = 'fff'
    
    return dynamic

# velocity에 대한 new_dynamic 컬럼 추가 함수

In [3]:
def add_new_dynamic_to_dataframe(input_file, target_file):
    """
    비어있는 dynamic 기호를 추가한다.
    1. note_on에 대한 dynamic 기호만 반영함.
        - dynamic 추가 방법
        1. 피아노 0.1초마다 note_on_status 값을 추출한다.
        2. 0.1 초에 해당하는 note_on 항목 중 velocity의 최대 값에 기반해 기호를 추출함.
    """
    input_file_note_status = [[{'note': x}, {'status': False}, {'velocity' : 0}] for x in range(128)]
    target_file_note_status = [[{'note': x}, {'status': False}, {'velocity' : 0}] for x in range(128)]

    input_dynamic_list = []
    target_dynamic_list = []
    
    # 인풋 데이터에 대해서 수행.
    for index in range(len(input_file)):

            target_temp_data = []

            notes_input = eval(input_file.at[index, 'note']) # 문자열을 -> 숫자로
            msg_type_input = eval(input_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
            velocity = eval(input_file.at[index, 'velocity'])

            temp = list(zip(notes_input, msg_type_input, velocity)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

            # 피아노 활성화 값 추적 
            for data in temp:
                if data[1] == 'note_on':
                    input_file_note_status[data[0]][1]['status'] = True
                    input_file_note_status[data[0]][2]['velocity'] = data[2]
                elif data[1] == 'note_off':
                    input_file_note_status[data[0]][1]['status'] = False
                    input_file_note_status[data[0]][2]['velocity'] = data[2]

            velocity_list = []

            for on_data in input_file_note_status:
                if on_data[1]['status'] == True:
                    velocity_list.append(({'note': on_data[0]['note']}, {'velocity' : on_data[2]['velocity']}))

            max_velocity = 0 # 최대 값 찾기

            for velocity in velocity_list:
                # 최대값 찾기
                if max_velocity < velocity[1]['velocity']:
                    max_velocity = velocity[1]['velocity']
            dynamic = distribute_dynamic(max_velocity)
            input_dynamic_list.append(dynamic)
            
    
    # 타겟 데이터에 대해 수행
    for index in range(len(target_file)):


            notes_input = eval(target_file.at[index, 'note']) # 문자열을 -> 숫자로
            msg_type_input = eval(target_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
            velocity = eval(target_file.at[index, 'velocity'])

            temp = list(zip(notes_input, msg_type_input, velocity)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

            # 피아노 활성화 값 추적 
            for data in temp:
                if data[1] == 'note_on':
                    target_file_note_status[data[0]][1]['status'] = True
                    target_file_note_status[data[0]][2]['velocity'] = data[2]
                elif data[1] == 'note_off':
                    target_file_note_status[data[0]][1]['status'] = False
                    target_file_note_status[data[0]][2]['velocity'] = data[2]

            velocity_list = []

            for on_data in target_file_note_status:
                if on_data[1]['status'] == True:
                    velocity_list.append(({'note': on_data[0]['note']}, {'velocity' : on_data[2]['velocity']}))

            max_velocity = 0 # 최대 값 찾기

            for velocity in velocity_list:
                # 최대값 찾기
                if max_velocity < velocity[1]['velocity']:
                    max_velocity = velocity[1]['velocity']
            dynamic = distribute_dynamic(max_velocity)
            target_dynamic_list.append(dynamic)


    target_file.insert(loc=5, column='new_dynamic', value=target_dynamic_list)
    input_file.insert(loc=5, column='new_dynamic', value=input_dynamic_list)
    

In [4]:
add_new_dynamic_to_dataframe(input_file, target_file)

# 비교 알고리즘

In [11]:
input_file.tail()

,sec,msg_type,channel,note,velocity,new_dynamic,dynamic,accent,count,main_vol,depth,pedal,pan
1201,120.1,[],[],[],[],mp,NaN,0,0,0,0,0,0
1202,120.2,[],[],[],[],mp,NaN,0,0,0,0,0,0
1203,120.3,[],[],[],[],mp,NaN,0,0,0,0,0,0
1204,120.4,"['note_off', 'note_off']","[0, 0]","[64, 64]","[0, 0]",p,ppp,0,2,0,0,0,0
1205,120.5,[],[],[],[],p,NaN,0,0,0,0,0,0


In [10]:
target_file.tail()

,sec,msg_type,channel,note,velocity,new_dynamic,dynamic,accent,count,main_vol,depth,pedal,pan
1209,120.9,[],[],[],[],ppp,NaN,0,0,0,0,0,0
1210,121.0,[],[],[],[],ppp,NaN,0,0,0,0,0,0
1211,121.1,[],[],[],[],ppp,NaN,0,0,0,0,0,0
1212,121.2,[],[],[],[],ppp,NaN,0,0,0,0,0,0
1213,121.3,[],[],[],[],ppp,NaN,0,0,0,0,0,0


## 기존 셈유림 변화 일관성 알고리즘

In [7]:
def calculate_dynamic_change_consistency(df_input, df_target):
    dynamic_series_input = df_input['new_dynamic']
    dynamic_series_target = df_target['new_dynamic']
    
    dynamic_states_input = zip(dynamic_series_input, dynamic_series_input.shift(-1))
    dynamic_states_target = zip(dynamic_series_target, dynamic_series_target.shift(-1))
    

    # 상태 바뀌는 부분 빈도수 계산
    dynamic_counter_input = pd.Series(dynamic_states_input).value_counts().to_dict()
    dynamic_counter_target = pd.Series(dynamic_states_target).value_counts().to_dict()
    
    
    # 입력 파일과 정답 파일에서 공통으로 나타나는 공통 패턴 찾기
    common_patterns = {state: min(dynamic_counter_input.get(state, 0), dynamic_counter_target.get(state, 0)) for state in set(dynamic_counter_input) | set(dynamic_counter_target)}
    # 일관성 백분율 계산
    total_common_patterns = sum(common_patterns.values())
    total_input_patterns = sum(dynamic_counter_input.values())

    
    
    # 100 이상이면 100%로 그냥 출력
    dynamic_change_consistency_percentage = min(100, (total_common_patterns / total_input_patterns) * 100)

    return f'{dynamic_change_consistency_percentage:.2f}%'

In [9]:
dynamic_change_score = calculate_dynamic_change_consistency(input_file, target_file)
print(f'셈여림 변화 일관성: {dynamic_change_score}')

셈여림 변화 일관성: 32.26%
